In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
! pip install -r requirement.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirement.txt'


In [3]:
%cd '/content/drive/MyDrive/content/pytorch-rotation-decoupled-detector/'

/content/drive/MyDrive/content/pytorch-rotation-decoupled-detector


In [4]:
import torch

from collections import Counter
from utils.box.bbox import decode
from utils.box.rbbox import rbbox_batched_nms as nms

In [ ]:
import pandas as pd

In [ ]:
df1 = pd.read_csv('yolo_1.csv')
df2 = pd.read_csv('yolo_2.csv')

In [ ]:
df1

,File,Confidence,X1,Y1,X2,Y2,X3,Y3,X4,Y4
0,OBJ00607,0.882727,128.000000,336.000000,147.000320,336.000000,147.000320,359.000064,128.000000,359.000064
1,OBJ00607,0.873315,376.000512,375.000064,376.000512,359.999488,394.999808,359.999488,394.999808,375.000064
2,OBJ00607,0.867435,45.802409,534.603159,57.296276,500.101539,95.397070,512.794434,83.903203,547.296055
3,OBJ00607,0.862871,309.412137,416.352500,288.705918,411.176276,296.352394,380.588413,317.058613,385.764636
4,OBJ00607,0.861346,238.396394,192.802672,215.897989,185.298083,224.804312,158.597346,247.302717,166.101935
...,...,...,...,...,...,...,...,...,...,...
7785,OBJ04999,0.160211,538.000384,211.999744,533.862190,207.344011,545.469156,197.027294,549.607350,201.683028
7786,OBJ04999,0.140651,405.051595,388.201303,434.827305,389.052411,433.097332,449.574919,403.321622,448.723811
7787,OBJ04999,0.130729,567.000064,977.999872,540.000256,977.999872,540.000256,615.999488,567.000064,615.999488
7788,OBJ04999,0.129565,469.768877,274.153672,466.077048,268.614983,482.232108,257.846781,485.923937,263.385470


In [ ]:
combine_df = pd.concat((df1, df2), axis = 0)
print(len(df1), len(df2))
df = combine_df

7790 5832


In [ ]:
combine_df

,File,Confidence,X1,Y1,X2,Y2,X3,Y3,X4,Y4
0,OBJ00607,0.882727,128.000000,336.000000,147.000320,336.000000,147.000320,359.000064,128.000000,359.000064
1,OBJ00607,0.873315,376.000512,375.000064,376.000512,359.999488,394.999808,359.999488,394.999808,375.000064
2,OBJ00607,0.867435,45.802409,534.603159,57.296276,500.101539,95.397070,512.794434,83.903203,547.296055
3,OBJ00607,0.862871,309.412137,416.352500,288.705918,411.176276,296.352394,380.588413,317.058613,385.764636
4,OBJ00607,0.861346,238.396394,192.802672,215.897989,185.298083,224.804312,158.597346,247.302717,166.101935
...,...,...,...,...,...,...,...,...,...,...
5827,OBJ04999,0.126460,435.999744,240.000000,442.999808,240.000000,442.999808,270.999552,435.999744,270.999552
5828,OBJ04999,0.111606,551.999488,508.000256,576.000000,508.000256,576.000000,538.999808,551.999488,538.999808
5829,OBJ04999,0.110671,495.297326,602.952601,456.719848,603.129459,455.206807,273.095776,493.784285,272.918917
5830,OBJ04999,0.109824,52.599879,11.799984,44.999948,7.999873,50.800136,-3.600057,58.400067,0.200054


In [ ]:

import sys

sys.path.append('.')

import os
import tqdm
import torch
import numpy as np

from collections import defaultdict
from torch.utils.data import DataLoader

from data.aug import ops
from data.aug.compose import Compose
from data.dataset import DOTA

from model.rdd import RDD
from model.backbone import resnet

from utils.box.bbox_np import xywha2xy4, xy42xywha
from utils.box.rbbox_np import rbbox_batched_nms
from utils.parallel import CustomDetDataParallel

In [ ]:
# ret_raw = {}

# for i in range(len(df)):
#   X = df.iloc[i]
#   fname = X['File']
#   if fname not in ret_raw:
#     ret_raw[fname] = []
#   scores = np.array([X['Confidence']])
#   xy4 = np.array([[X['X1'], X['Y1']], [X['X2'], X['Y2']], [X['X3'], X['Y3']], [X['X4'], X['Y4']]])
#   xywha = xy42xywha(xy4)
#   # xy4 = np.array([[X['X1'], X['X2'], X['X3'], X['X4']], [X['Y1'], X['Y2'], X['Y3'], X['Y4']]])
#   ret_raw[fname].append([xywha, scores, np.array([0])])]

ret_raw = {}
for name in df['File'].unique():
  X = df[df['File'] == name]
  scores = np.array(X['Confidence'])
  bboxes = np.array([[X['X1'], X['Y1']], [X['X2'], X['Y2']], [X['X3'], X['Y3']], [X['X4'], X['Y4']]])
  t = np.transpose(bboxes, (2, 0, 1))
  xywha = np.stack([xy42xywha(bbox) for bbox in t])
  labels = np.array([0 for i in range(len(scores))])
  ret_raw[name] = [xywha, scores, labels]

In [ ]:
ret = []
nms_thresh = 0.3
for fname, value in ret_raw.items():
  bboxes, scores, labels = zip(value)
  bboxes = np.concatenate(list(bboxes))
  scores = np.concatenate(list(scores))
  labels = np.concatenate(list(labels))
  keeps = rbbox_batched_nms(bboxes, scores, labels, nms_thresh)
  ret.append([fname, [bboxes[keeps], scores[keeps], labels[keeps]]])

In [ ]:
print('converting to submission format...')
ret_save = defaultdict(list)
for fname, (bboxes, scores, labels) in ret:
    for bbox, score, label in zip(bboxes, scores, labels):
        bbox = xywha2xy4(bbox).ravel()
        line = '%s %.12f %.1f %.1f %.1f %.1f %.1f %.1f %.1f %.1f' % (fname, score, *bbox)
        ret_save['container'].append(line)

converting to submission format...


In [ ]:
print('saving...')
dir_save = '/content/drive/MyDrive/content/pytorch-rotation-decoupled-detector/'
os.makedirs(os.path.join(dir_save, 'submission'), exist_ok=True)
for name, dets in ret_save.items():
    with open(os.path.join(dir_save, 'submission', 'Task%d_%s.txt' % (3, name)), 'wt') as f:
        f.write('\n'.join(dets))

print('finished')

saving...
finished


### nms_thresh 설정
#(1) 기존
1. nms_thresh(0.45): 8945
2. nms_thresh(0.3): 7375
3. nms_thresh(0.2): 6764
#(2) 36000_with_DOTA_backbone.csv
1. nms_thresh(0.2): 8769
2. nms_thresh(0.1): 7989
# (3) combine
1. nms_thresh(0.2): 14448
2. nms_thresh(0.05): 12179


# (13622, 9413)

In [ ]:
len(df), len(ret_save['container'])

(13622, 9413)

In [ ]:
tmp = ret_save['container']
data = [tmp[i].split(' ') for i in range(len(tmp))]
print(data)

[['OBJ00607', '0.884701000000', '128.0', '336.0', '147.0', '336.0', '147.0', '356.0', '128.0', '356.0'], ['OBJ00607', '0.873315000000', '395.0', '360.0', '395.0', '375.0', '376.0', '375.0', '376.0', '360.0'], ['OBJ00607', '0.867435000000', '95.4', '512.8', '83.9', '547.3', '45.8', '534.6', '57.3', '500.1'], ['OBJ00607', '0.862871000000', '296.4', '380.6', '317.1', '385.8', '309.4', '416.4', '288.7', '411.2'], ['OBJ00607', '0.861346000000', '224.8', '158.6', '247.3', '166.1', '238.4', '192.8', '215.9', '185.3'], ['OBJ00607', '0.857065000000', '123.0', '340.0', '123.0', '351.0', '108.0', '351.0', '108.0', '340.0'], ['OBJ00607', '0.856901000000', '327.0', '348.0', '327.0', '363.0', '308.0', '363.0', '308.0', '348.0'], ['OBJ00607', '0.855187000000', '187.5', '196.9', '177.3', '224.3', '112.8', '200.3', '123.0', '172.9'], ['OBJ00607', '0.854925000000', '172.0', '600.0', '187.0', '600.0', '187.0', '631.0', '172.0', '631.0'], ['OBJ00607', '0.854080000000', '337.3', '290.0', '356.2', '296.8', 

In [ ]:
df2 = pd.DataFrame(data)
df2.columns = df.columns
df2.to_csv(dir_save + '/submission/combine_model.csv', index=False);
df2

,File,Confidence,X1,Y1,X2,Y2,X3,Y3,X4,Y4
0,OBJ00607,0.884701000000,128.0,336.0,147.0,336.0,147.0,356.0,128.0,356.0
1,OBJ00607,0.873315000000,395.0,360.0,395.0,375.0,376.0,375.0,376.0,360.0
2,OBJ00607,0.867435000000,95.4,512.8,83.9,547.3,45.8,534.6,57.3,500.1
3,OBJ00607,0.862871000000,296.4,380.6,317.1,385.8,309.4,416.4,288.7,411.2
4,OBJ00607,0.861346000000,224.8,158.6,247.3,166.1,238.4,192.8,215.9,185.3
...,...,...,...,...,...,...,...,...,...,...
9408,OBJ04754,0.320901000000,647.0,316.0,647.0,319.0,637.0,319.0,637.0,316.0
9409,OBJ04754,0.219407000000,50.0,4.0,50.0,11.0,36.0,11.0,36.0,4.0
9410,OBJ04754,0.211508000000,663.0,284.0,663.0,291.0,652.0,291.0,652.0,284.0
9411,OBJ04864,0.410987000000,637.0,440.0,637.0,447.0,628.0,447.0,628.0,440.0


In [6]:
cd '/content/drive/MyDrive/content/pytorch-rotation-decoupled-detector/run/dota'

/content/drive/MyDrive/content/pytorch-rotation-decoupled-detector/run/dota


In [7]:
! chmod 777 '/content/drive/MyDrive/content/pytorch-rotation-decoupled-detector/images/vali/valid_images'

In [8]:
# ! python visualize_label.py --image "/content/drive/MyDrive/content/pytorch-rotation-decoupled-detector/images/vali/valid_images" --label "/content/drive/MyDrive/content/pytorch-rotation-decoupled-detector/60000_wo_backbone.csv"
! python visualize_label.py --image "/content/drive/MyDrive/content/pytorch-rotation-decoupled-detector/images/vali/valid_images" --label "/content/drive/MyDrive/content/pytorch-rotation-decoupled-detector/submission/Task3_container.txt" --comma "no"

OBJ01347_PS3_K3_NIA0092.png_with_labels.jpg
OBJ01357_PS3_K3_NIA0092.png_with_labels.jpg
OBJ01527_PS3_K3_NIA0095.png_with_labels.jpg
OBJ01675_PS3_K3_NIA0102.png_with_labels.jpg
OBJ01701_PS3_K3_NIA0102.png_with_labels.jpg
OBJ02240_PS3_K3_NIA0117.png_with_labels.jpg
OBJ02546_PS3_K3_NIA0124.png_with_labels.jpg
OBJ03368_PS3_K3A_NIA0137.png_with_labels.jpg
OBJ00607_PS3_K3_NIA0086.png_with_labels.jpg
OBJ03369_PS3_K3A_NIA0137.png_with_labels.jpg
OBJ03391_PS3_K3A_NIA0137.png_with_labels.jpg
OBJ03424_PS3_K3A_NIA0139.png_with_labels.jpg
OBJ03425_PS3_K3A_NIA0139.png_with_labels.jpg
OBJ03426_PS3_K3A_NIA0139.png_with_labels.jpg
OBJ03445_PS3_K3A_NIA0139.png_with_labels.jpg
OBJ03477_PS3_K3A_NIA0142.png_with_labels.jpg
OBJ03481_PS3_K3A_NIA0143.png_with_labels.jpg
OBJ03540_PS3_K3A_NIA0148.png_with_labels.jpg
OBJ03563_PS3_K3A_NIA0150.png_with_labels.jpg
OBJ03570_PS3_K3A_NIA0150.png_with_labels.jpg
OBJ03572_PS3_K3A_NIA0150.png_with_labels.jpg
OBJ03579_PS3_K3A_NIA0151.png_with_labels.jpg
OBJ03594_PS3_K3A_N